## To plot in Qt window to add interactivity

In [ ]:
%pylab qt

## To read .csv file:

In [ ]:
import csv

#to change working directory
#import os
#os.chdir(r'D:\!Data\2015_09_22')

filenabe = 'tek00017.csv'

#only for 2 coloms
x = []
y = []
with open(filename, 'r') as csvfile:
    a = csv.reader(csvfile)
    for line in a:
        x.append(float(line[0]))
        y.append(float(line[1]))        
#plot(x,y)

## To find and fit peaks:

In [ ]:
# additional description is available in lasercooling.ru
# updated on 28.11.2015 
# handling border limitation, filtering, pick number plot

# x, y - data to prosess, should be initialize befor

# I assume that  command below was executed, qt - to show plots in separate window, use 'inline' instead to plot here 
#%pylab qt
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter
from inspect import getargspec

# Gaussian function
def gaussian(x,N,x0,sigma, background):
    """Returns value of a 1D-gaussian with the given parameters ,N,x0,sigma, background"""
    #from numpy import sqrt,pi,exp
    return N / (sigma * sqrt(pi)) * exp(-(x - x0)**2/(sigma**2)) + background

# lorentz function
def lorenz(x, a, x0, g, bg):
    return a * g/2 / ((x-x0)**2 + g**2/4) + bg

# If filtering is needed
y = gaussian_filter(y,20)

# threshold value to find piks and then to estimate width
thr = 0.04 #max(y)/2

# construct list of indexis y[index] > threshold
n1 = [x for (x,val) in enumerate(y) if val > thr] 
#print(n1)

# construct coordinates of beggining and ending of the pick based on threshold value
coords = []
for num,elem in enumerate(n1):
    if num == 0:   # iteration start
        st = 0
    elif elem - n1[num-1] > 400:    # if jump to next pick, number specifies minimum distance between peaks
        coords.append((n1[st],n1[num-1]))
        st = num
    elif  num == len(n1)-1:       # if came to list end
        coords.append((n1[st],n1[num]))
# print(coords)        

# fitting of individual peak in the neghboring of n * width and plotting them
n = 8     #what range to use to find fit
plot(x,y)
popts = []
# fit_func = lorenz
fit_func = gaussian
print('Fit parameters:',getargspec(fit_func)[0][1:])
for num,(lw,up) in enumerate(coords):
    width = abs(up - lw)
    sls = slice(max(0,lw - n*width), min(up + n*width,len(x)))
    popt, pcov = curve_fit(fit_func, x[sls], y[sls],p0=(0.001, (x[up]+x[lw])/2, (x[up]-x[lw]),0.023))
    plot(x[sls],fit_func(x[sls],*popt))
    popts.append(popt)
    annotate(str(num),xy=(popt[1],0))
    print(num, '---->', popt)